In [1]:
import numpy as np
import matplotlib.pyplot as plt
from abg_python.snapshot_utils import openSnapshot
from abg_python.cosmo_utils import load_AHF
from itk.itk import inrange, loadpickle, pickle_save_dict
from numba import njit

In [2]:
exec(open('scripts/halo_analysis_scripts.py').read())

In [3]:
# Quest (m13 no feedback)
for sim in ['h206', 'h29', 'h113', 'h2']:
    simname = sim + '_HR_sn1dy300ro100ss'
    snapnum = 187
    p0 = load_p0(f'/projects/b1026/anglesd/FIRE/{simname}', snapnum, ahf_path=f'/projects/b1026/halo_files/anglesd_m13/{simname}')
    print(sim, p0['Rvir'])
    profiles(p0, outfile=f'data/{simname}_{snapnum}.pkl');

/projects/b1026/anglesd/FIRE/h206_HR_sn1dy300ro100ss/snapdir_187/snapshot_187.0.hdf5
This is a cosmological snapshot... converting to physical units
/projects/b1026/anglesd/FIRE/h206_HR_sn1dy300ro100ss/snapdir_187/snapshot_187.1.hdf5
/projects/b1026/anglesd/FIRE/h206_HR_sn1dy300ro100ss/snapdir_187/snapshot_187.2.hdf5
/projects/b1026/anglesd/FIRE/h206_HR_sn1dy300ro100ss/snapdir_187/snapshot_187.3.hdf5
/projects/b1026/anglesd/FIRE/h206_HR_sn1dy300ro100ss/snapdir_187/snapshot_187.0.hdf5
This is a cosmological snapshot... converting to physical units
/projects/b1026/anglesd/FIRE/h206_HR_sn1dy300ro100ss/snapdir_187/snapshot_187.1.hdf5
/projects/b1026/anglesd/FIRE/h206_HR_sn1dy300ro100ss/snapdir_187/snapshot_187.2.hdf5
/projects/b1026/anglesd/FIRE/h206_HR_sn1dy300ro100ss/snapdir_187/snapshot_187.3.hdf5
Loading redshift 1.799999681531304
h206 167.4216218297942
/projects/b1026/anglesd/FIRE/h29_HR_sn1dy300ro100ss/snapdir_187/snapshot_187.0.hdf5
This is a cosmological snapshot... converting to p